# Full subsidence curve, Trout Creek Sequence, McCoy Creek Group, and overlying Pz strata
* * *
* * * 

## Build SubsidenceChron by module, modified from Zhang et al., 2023
* * * 

In [ ]:
## --- Custom objects for holding SubsidenceChron data

    # A type of object to hold data about the stratigraphy
    mutable struct StratData
        Lithology::Array{String}
        Thickness::Array{Float64,1}
    end

    function NewStratData(nLayers)
        strat = StratData(
            fill("", nLayers),
            fill(NaN,nLayers),
        )
        return strat
    end
    export NewStratData

    mutable struct WaterDepth
        DepthID::Array{String}
        Thickness::Array{Float64,1}
    end

    function NewWaterDepth(wd_nLayers)
        wd = WaterDepth(
            fill("", wd_nLayers),
            fill(NaN, wd_nLayers),
        )
        return wd
    end
    export NewWaterDepth

    # A type of object to hold data about the thermal subsidence parameters
    mutable struct ThermalSubsidenceParameters
        Param::Array{Float64,1}
        Sigma::Array{Float64,1}
    end

    function NewThermalSubsidenceParameters()
        therm = ThermalSubsidenceParameters(
            fill(NaN,2),
            fill(NaN,2),
        )
        return therm
    end
    export NewThermalSubsidenceParameters

    struct SubsidenceStratAgeModel
        Height::Array{Float64,1}
        Age::Array{Float64,1}
        Age_sigma::Array{Float64,1}
        Age_Median::Array{Float64,1}
        Age_025CI::Array{Float64,1}
        Age_975CI::Array{Float64,1}
        Beta::Array{Float64,1}
        Beta_sigma::Array{Float64,1}
        Beta_Median::Array{Float64,1}
        Beta_025CI::Array{Float64,1}
        Beta_975CI::Array{Float64,1}
        T0::Array{Float64,1}
        T0_sigma::Array{Float64,1}
        T0_Median::Array{Float64,1}
        T0_025CI::Array{Float64,1}
        T0_975CI::Array{Float64,1}
    end

In [ ]:
## --- Subsidence parameters

function subsidenceparams(lithology::AbstractString)
    if lithology == "Shale"
        # c has a lower bound of 0 b/c porosity at depth should not be greater than porosity at surface
        c_dist = truncated(Normal(0.51, 0.15), 0, Inf)
        # ϕ₀ has a lower bound of 0 and an upper bound of 1 b/c of the definition of porosity
        ϕ₀_dist = truncated(Normal(0.63, 0.15), 0, 1)
        ρg = 2720
    elseif lithology == "Siltstone"
        c_dist = truncated(Normal(0.39, 0.1), 0, Inf)
        ϕ₀_dist = truncated(Normal(0.56, 0.1), 0, 1)
        ρg = 2640 # Might need to update this number
    elseif lithology == "Sandstone"
        c_dist = truncated(Normal(0.27, 0.1), 0, Inf)
        ϕ₀_dist = truncated(Normal(0.49, 0.1), 0, 1)
        ρg = 2650
    elseif lithology == "Chalk"
        c_dist = truncated(Normal(0.71, 0.15), 0, Inf)
        ϕ₀_dist = truncated(Normal(0.7, 0.15), 0, 1)
        ρg = 2710
    elseif lithology == "Limestone"
        c_dist = truncated(Normal(0.6, 0.2), 0, Inf)
        ϕ₀_dist = truncated(Normal(0.4, 0.17), 0, 1)
        ρg = 2710
    elseif lithology == "Dolostone"
        c_dist = truncated(Normal(0.6, 0.2), 0, Inf)
        ϕ₀_dist = truncated(Normal(0.2, 0.1), 0, 1)
        ρg = 2870
    elseif lithology == "Anhydrite"
        c_dist = truncated(Normal(0.2, 0.1), 0, Inf)
        ϕ₀_dist = truncated(Normal(0.05, 0.05), 0, 1)
        ρg = 2960
    elseif lithology == "Quartzite"
        c_dist = truncated(Normal(0.3, 0.1), 0, Inf)
        ϕ₀_dist = truncated(Normal(0.2, 0.1), 0, 1)
        ρg = 2650
     elseif lithology == "Diabase"
        c_dist = truncated(Normal(0.65, 0.1), 0, Inf)
        ϕ₀_dist = truncated(Normal(0.285, 0.1), 0, 1)
        ρg = 2960
    elseif lithology == "Rhyolite"
        c_dist = truncated(Normal(0.65, 0.1), 0, Inf)
        ϕ₀_dist = truncated(Normal(0.275, 0.1), 0, 1)
        ρg = 2510
    elseif lithology == "Diamictite"
        c_dist = truncated(Normal(0.51, 0.15), 0, Inf)
        ϕ₀_dist = truncated(Normal(0.63, 0.15), 0, 1)
        ρg = 2720
    elseif lithology == "Conglomerate"
        c_dist = truncated(Normal(0.51, 0.15), 0, Inf)
        ϕ₀_dist = truncated(Normal(0.63, 0.15), 0, 1)
        ρg = 2720
    elseif lithology == "Breccia"
        c_dist = truncated(Normal(0.51, 0.15), 0, Inf)
        ϕ₀_dist = truncated(Normal(0.63, 0.15), 0, 1)
        ρg = 2720
    elseif lithology == "Basalt"
        c_dist = truncated(Normal(0.65, 0.1), 0, Inf)
        ϕ₀_dist = truncated(Normal(0.05, 0.1), 0, 1)
        ρg = 2980
    elseif lithology == "Andesite"
        c_dist = truncated(Normal(0.65, 0.1), 0, Inf)
        ϕ₀_dist = truncated(Normal(0.095, 0.1), 0, 1)
        ρg = 2650
    else # fallback, if unknown
        @warn "lithology $lithology not recognized, using default porosity parameters"
        c_dist = truncated(Normal(0.5, 0.3), 0, Inf)
        ϕ₀_dist = Uniform(0, 1)
        ρg = 2700
    end
    return c_dist, ϕ₀_dist, ρg
end
function subsidenceparams(lithology::Vector{<:AbstractString})
    # porosity depth coefficient(c)
    c_dist = similar(lithology, Distribution)
    # surface porosity (ϕ₀)
    ϕ₀_dist = similar(lithology, Distribution)
    # sediment grain density (ρg)
    ρg = similar(lithology, Float64)

    # Find the correct c, ϕ₀, and ρg for each layer based on lithology
    for i in eachindex(lithology)
        c_dist[i], ϕ₀_dist[i], ρg[i] = subsidenceparams(lithology[i])
    end
    return c_dist, ϕ₀_dist, ρg
end

In [ ]:
import Pkg; Pkg.add("ProgressMeter")
using StatGeochem, Distributions, Plots, Statistics, StatsBase, Chron
using ProgressMeter: @showprogress, Progress, update!

In [ ]:
# Define the decompaction function
function decompact!(yₚ, y, ϕ₀, c, n, m; niterations = 10)
    # y[n+1] = y₂ = present-day depth for base of layer n
    # y[n] = y₁ = present-day depth for top of layer n
    δy = y[n+1] - y[n]
    # yₚ[n-m+2] = y₂' = depth for base of layer n during decompaction (at t=tₘ)
    # yₚ[n-m+1] = y₁' = depth for top of layer n during decompaction (at t=tₘ)
    yₚ[n-m+2] = yₚ[n-m+1] + δy
    # In this for loop, the initial approximatation for the thickness of a given layer is the present-day thickness of that layer (calculated above)
    # The true thickness is then reached by running the decompaction equation multiple times (niterations)
    e₁ = exp(-c*yₚ[n-m+1])
    e₃₄ = exp(-c*y[n]) - exp(-c*y[n+1])
    @inbounds for idx=1:niterations
        # yₚ[n-m+2] = yₚ[n-m+1] + δy + ϕ₀/c * ((exp(-c*yₚ[n-m+1]) - exp(-c*yₚ[n-m+2])) - (exp(-c*y[n]) - exp(-c*y[n+1])))
        yₚ[n-m+2] = yₚ[n-m+1] + δy + ϕ₀/c * ((e₁ - exp(-c*yₚ[n-m+2])) - e₃₄)
    end
end

In [ ]:
# Define the DecompactBackstrip function; includes parameterization of water depth distributions
function DecompactBackstrip(strat::StratData, wd::WaterDepth, nsims, res)

    # Import data from csv and assign parameters for each lithology
        lithology_inputs = strat.Lithology
        height_inputs = cumsum([0; strat.Thickness])
        nlayer_input = length(strat.Thickness)
        model_strat_heights = [0:res:maximum(height_inputs);]
        model_nlayer = length(model_strat_heights)-1

    # Allocate parameters as distributions; each element/distribution represents a layer
        # c: porosity depth coefficient, ϕ₀: surface porosity, ρg: sediment grain density
        c_dist, ϕ₀_dist, ρg = subsidenceparams(lithology_inputs)

    # Prep for decompaction and backstripping MC
        # Define parameters for decompaction (number of simulations; water and mantle densities)
        ρw = 1000
        ρm = 3330

        # Allocate depth matricies (rows are strat horizons, columns are timesteps)
        # Decompacted depth
        Y = fill(1E-18, (model_nlayer+1, model_nlayer+1))

        # Allocate decompaction parameter matricies (for all modeled layers with a resolution = res)
        c_highres = Array{Float64,1}(undef, model_nlayer)
        ϕ₀_highres = Array{Float64,1}(undef, model_nlayer)
        ρg_highres = Array{Float64,1}(undef, model_nlayer)

        # Allocate paleo water depth matrix (for all modeled layers with a resolution = res)
        paleo_wd_dist = Array{Float64,1}(undef, nsims)

        # Allocate porosity, density and tectonic subsidence matricies
        # Porosity of a strat unit at any depth
        ϕ_avg = fill(1E-18, (model_nlayer, model_nlayer+1))
        # Bulk density of a single layer
        ρ_bulk = fill(1E-18, (model_nlayer, model_nlayer+1))
        # Intermediate step - bulk density*thickness of a single layer
        m_bulk = fill(1E-18, (model_nlayer, model_nlayer+1))
        # Bulk density of the entire column
        ρ_bulk_column = Array{Float64,1}(undef, model_nlayer+1)
        # Tectonic subsidence # this is the only one that will need to propagate outside of the loop
        Sₜ_km = Array{Float64,2}(undef, model_nlayer+1, nsims)

    # MC for decompaction and backstripping
        # Visualize the progress in terminal
        print("Decompaction and Backstripping: ", nsims, " steps\n")
        pgrs = Progress(nsims, desc="Decompaction and Backstripping...")
        pgrs_interval = ceil(Int,10)
        c = rand.(c_dist)
        ϕ₀ = rand.(ϕ₀_dist)

        for sim = 1:nsims
            # Randomly select c and ϕ₀ vectors from the distributions for each input layer
            @. c = rand(c_dist)
            @. ϕ₀ = rand(ϕ₀_dist)

            # Propagate these selections to every model layers; all model layers from the same input layer get the same c and ϕ₀ values
            @inbounds for i = 1:nlayer_input
                for j = 1:model_nlayer
                    if model_strat_heights[j+1]>height_inputs[i]
                        c_highres[j]=c[i]
                        ϕ₀_highres[j]=ϕ₀[i]
                        ρg_highres[j]=ρg[i]
                    end
                end
            end

            # Fill the first column with modern observed values (present-day depths)
            Y[:,1] .= model_strat_heights
            # Fill the first row with zeros
            Y[1,:] .= 0

            # Decompact
            # i = time steps during decompaction, which runs from 2 to layer_count b/c column 1 is present day
            # j = layer number, which runs from i to layer_count b/c the ith column begins with y₁' of layer i
            for i = 2:model_nlayer
                for j = i:model_nlayer
                    decompact!(view(Y,:,i), model_strat_heights, ϕ₀_highres[j], c_highres[j], j, i)
                end
            end

            # Import water depth correction data (water depth categories based on sedimentological evidences)
            wd_id_inputs = wd.DepthID
            wd_height_inputs = cumsum([0; wd.Thickness])
            wd_nlayer_input = length(wd.Thickness)

            # Allocate matrix for paleo-water depth distributions (for all water depth input layers)
            paleo_wd_dist = Array{Distribution,1}(undef, wd_nlayer_input)
            # Define the water depth distributions for all input layers based on their water depth classifications
            for i = 1:wd_nlayer_input
                if wd_id_inputs[i] == "Exposure"
                    paleo_wd_dist[i] = Uniform(0, 0.00001)
                elseif wd_id_inputs[i] == "FWWB"
                    paleo_wd_dist[i] = Uniform(0.005,0.050)
                elseif wd_id_inputs[i] == "SWB"
                    paleo_wd_dist[i] = Uniform(0.03,0.2)
                elseif wd_id_inputs[i] == "BWB"
                    paleo_wd_dist[i] = Uniform(0.15,1)
                elseif wd_id_inputs[i] == "BWBS"
                    paleo_wd_dist[i] = Uniform(0.15,0.3)
                end
            end
        

            # Randomly select paleo water depth value (from the distributions) for each input layer
            paleo_wd = rand.(paleo_wd_dist)
            # Allocate paleo_wd matrix for all modeled layers with a resolution = res
            paleo_wd_highres = Array{Float64,1}(undef, model_nlayer+1)

            # Setup this matrix - paleo water depth for the first (topmost) modeled layer should be the same as the paleo water depth for the first input layer
            paleo_wd_highres[1] = paleo_wd[1]
            # Propagate these selections to every model layers; all model layers from the same input layer get the same paleo water depth
            for i = 1:wd_nlayer_input
                for j = 1:model_nlayer+1
                    if model_strat_heights[j]>wd_height_inputs[i]
                        paleo_wd_highres[j]=paleo_wd[i]
                    end
                end
            end

            # Remove the effect of sediment load - same indexing logic as the decompaction loop
            # i = time steps (columns) = 1:layer_count b/c need to calculate these parameters for the present day column/layers
            # j = layer number = i:layer_count b/c the ith column begins with y₁' of layer i
            @inbounds for i = 1:model_nlayer+1
                for j = i:model_nlayer
                    # Average porosity of all modeled layers
                    ϕ_avg[j-i+1,i] = (ϕ₀_highres[j]/c_highres[j])*(exp(-c_highres[j]*Y[j-i+1,i])-exp(-c_highres[j]*Y[j-i+2,i]))/(Y[j-i+2,i]-Y[j-i+1,i])
                    # Bulk density of all modeled layers
                    ρ_bulk[j-i+1,i] = ρw*ϕ_avg[j-i+1,i]+(1-ϕ_avg[j-i+1,i])*ρg_highres[j]
                    # Mass of all modeled layers
                    m_bulk[j-i+1,i] = (Y[j-i+2,i]-Y[j-i+1,i])*ρ_bulk[j-i+1,i]
                end
            end
            # (Mass = 0 for the last column, which represents the moment right before the deposition of the bottommost layer)
            m_bulk[:,end] .= 0

            # Backstripping calculations
            @inbounds for i = 1:model_nlayer+1
                # Bulk density of all the columns (aka bulk densities of the whole sediment column for all timesteps)
                # maximum([:,i]) = total depth of column at time step i
                ρ_bulk_column[i] = sum(view(m_bulk,:,i))/maximum(view(Y,:,i))
                # Tectonic subsidence for all timesteps; record results from all simulations
                Sₜ_km[i,sim] = Y[model_nlayer+2-i,i]*((ρm-ρ_bulk_column[i])/(ρm-ρw)).+paleo_wd_highres[i]
            end

            mod(sim,pgrs_interval)==0 && update!(pgrs, sim)
        end
        update!(pgrs,nsims)

        # Calculate summary statistics (mean and standard deviation)
        Sₜ = Sₜ_km .* 1000
        Sμ = nanmean(Sₜ, dim=2)
        Sσ = nanstd(Sₜ, dim=2)

    return Sₜ, Sμ, Sσ, model_strat_heights
end

# Import input data; run decompaction and backstripping 
* * * 

In [ ]:
## --- Part 1: Decompaction and Backstripping

    # # # # # # # # # # # Enter stratigraphic information here! # # # # # # # # # # # #
    # Import the data file (.csv)
    data_csv = importdataset("Input/McCoy_highres.csv",',')
    # Obtain stratigraphic info from the data file
    nLayers = length(data_csv["Thickness"])
    strat = NewStratData(nLayers)
    strat.Lithology          = data_csv["Lithology"]
    strat.Thickness         .= data_csv["Thickness"]
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

    # # # # # # # # OPTIONAL - Enter paleo water depth information here! # # # # # # # # 
    # Import the data file (.csv)
    wd_csv = importdataset("Input/McCoy_seq.csv", ',')
    # Obtain paleo water depth info from the data file
    wd_nLayers = length(wd_csv["Thickness"])
    wd = NewWaterDepth(wd_nLayers)
    wd.DepthID    = wd_csv["Type"]
    wd.Thickness .= wd_csv["Thickness"] 
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

    # # # # # # # # # # Configure MC model here! # # # # # # # # # #
    # Number of MC simulations
    nsims = 4000
    # Resolution for model horizons (in km)
    res = 0.01
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
    
    #SET FILEPATH#
    Path = "Output/Fullstrat_Adj/"
    # Run the decompaction and backstripping MC model with water depth
    
    @time (Sₜ, Sμ, Sσ, model_strat_heights) = DecompactBackstrip(strat, wd, nsims, res)

    # Code for storing and reading decompaction + backstripping results - will be useful when testing the age-depth modeling part of the model
    # Store results
    using DelimitedFiles
    writedlm(Path*"Mc_highres.csv", Sₜ, ",")
    writedlm(Path*"Mc_mu_highres.csv", Sμ)
    writedlm(Path*"Mc_sigma_highres.csv", Sσ)
    writedlm(Path*"Mc_highres.txt", Sₜ)
    writedlm(Path*"Mc_stratheight.csv",model_strat_heights,",")
    # Read results
    Sₜ = readdlm(Path*"Mc_highres.csv", ',', Float64)
    Sμ = readdlm(Path*"Mc_mu_highres.csv", ',', Float64)
    Sσ = readdlm(Path*"Mc_sigma_highres.csv", ',', Float64)

In [ ]:
# Plot results - tectonic subsidence in comparison with present day stratigraphic heights
p1 = plot(Sₜ[:,2:end], alpha = 0.01, label = "", yflip = true, color = "blue", fg_color_legend=:white)

    #plot!(p1, reverse((model_strat_heights)*1000), yflip = true, label = "Present-day thickness", color = "red")
plot!(Sμ, alpha = 1, yflip = true, xflip = true, label = "Tectonic subsidence, mean", color = "blue")
    savefig(p1, Path*"McCoy_DecompactBackstrip_higherres_SL.pdf")

display(p1)

## Build age-height model by module
* * * 
* * *

In [ ]:
using Statistics, StatsBase, DelimitedFiles, SpecialFunctions

In [ ]:
# Define the function that calculates the log likelihood of the thermal subsidence fit
function subsidence_ll(E₀, τ, model_St, model_St_sigma, model_t, beta_t0)
    @assert eachindex(model_St) == eachindex(model_St_sigma) == eachindex(model_t)
    β, T₀ = beta_t0
    ll = zero(float(eltype(model_St_sigma)))
    for i ∈ eachindex(model_t)
        # Calculate subsidence_model_heights given this unique smooth thermal subsidence model at each age in model_t
        x = (E₀*β/pi)*sin(pi/β)*(1 -exp(-(T₀ -model_t[i])/τ))
        mu = model_St[i]
        sigma = model_St_sigma[i]
        # Turn that into a log likelihood using some age uncertainty of the curve
        ll -= (x-mu)*(x-mu) / (2*sigma*sigma)
    end
    return ll

    return log_likelihood
end

In [ ]:
function SubsidenceStratMetropolis(smpl::ChronAgeData, config::StratAgeModelConfiguration, therm::ThermalSubsidenceParameters, model_strat_heights, Sμ, Sσ, beta_ip, t0_ip;
        subsidencebottom=minimum(smpl.Height),
        subsidencetop=maximum(smpl.Height)
    )

    # Run stratigraphic MCMC model
    print("Generating stratigraphic age-depth model...\n")

    # Define thermal subsidence model parameters
        y_litho= 125000
        ρ_mantle = 3330
        ρ_basinfill = 1000 # originally  ρ_water = 1000 by default in Zhang et al., 2023; this only works if you wish to model the behavior of a water-filled basin. 
        αᵥ = 3.28*10^(-5)
        T_mantle = 1333
        τ = 50 #Myr
        E₀ = (4*y_litho*ρ_mantle*αᵥ*T_mantle)/(pi^2*(ρ_mantle-ρ_basinfill))

    # Stratigraphic age constraints
        Age = copy(smpl.Age)::Array{Float64,1}
        Age_sigma = copy(smpl.Age_sigma)::Array{Float64,1}
        Height = copy(smpl.Height)::Array{Float64,1}
        Height_sigma = smpl.Height_sigma::Array{Float64,1} .+ 1E-9 # Avoid divide-by-zero issues
        Age_Sidedness = copy(smpl.Age_Sidedness)::Array{Float64,1} # Bottom is a maximum age and top is a minimum age
        (bottom, top) = extrema(Height)
        (youngest, oldest) = extrema(Age)
        dt_dH = (oldest-youngest)/(top-bottom)
        aveuncert = nanmean(Age_sigma)

    # Model configuration -- read from struct
        resolution = config.resolution
        bounding = config.bounding
        nsteps = config.nsteps
        burnin = config.burnin
        sieve = config.sieve
        model_heights = bottom:resolution:top

        if bounding>0
            # If bounding is requested, add extrapolated top and bottom bounds to avoid
            # issues with the stratigraphic markov chain wandering off to +/- infinity
            offset = round((top-bottom)*bounding/resolution)*resolution
            Age = [oldest + offset*dt_dH; Age; youngest - offset*dt_dH]
            Age_sigma = [nanmean(Age_sigma)/10; Age_sigma; nanmean(Age_sigma)/10]
            Height = [bottom-offset; Height; top+offset]
            Height_sigma = [0; Height_sigma; 0] .+ 1E-9 # Avoid divide-by-zero issues
            Age_Sidedness = [-1.0; Age_Sidedness; 1.0;] # Bottom is a maximum age and top is a minimum age
            model_heights = (bottom-offset):resolution:(top+offset)
        end

        active_height_t = bottom .<= model_heights .<= top
        npoints = length(model_heights)

    # STEP 1: calculate log likelihood of the modeled ages (and heights) in the initial proposal
        # Start with a linear fit as an initial proposal
        (a,b) = hcat(fill!(similar(Height), 1), Height) \ Age
        #b = -0.04
        model_ages = a .+ b .* collect(model_heights)

        # Calculate log likelihood of initial proposal
        # Proposals younger than age constraint are given a pass if Age_Sidedness is -1 (maximum age)
        # proposals older than age constraint are given a pass if Age_Sidedness is +1 (minimum age)
        sample_height = copy(Height)
        closest = findclosest(sample_height, model_heights)
        closest_model_ages = model_ages[closest]

        #Age_twosided = Array{Float64}(undef, count(==(0), Age))
        #Age_sigma_twosided = Array{Float64}(undef, count(==(0), Age))

        @inbounds for i=1:length(Age)
            if Age_Sidedness[i] == sign(closest_model_ages[i] - Age[i])
                closest_model_ages[i] = Age[i]
            end
            #if Age_sidedness[i] == 0
            #    for j = 1:length(Age_twosided)
            #        Age_twosided[j] = Age[i]
            #        Age_sigma_twosided[j] = Age_sigma[i]
            #    end
            #end
        end

        ll = normpdf_ll(Age, Age_sigma, closest_model_ages)
        ll += normpdf_ll(Height, Height_sigma, sample_height)

    # STEP 2: calculate log likelihood of the subsidence model parameters in the initial proposal
        # Define thermal subsidence model parameters
        ideal_subs_parameters = therm.Param
        ideal_subs_parameters_sigma = therm.Sigma

        # Initial proposal for subsidence parameters - randomly pick a set of values from the distribution
        subs_parameters = [ideal_subs_parameters[1]+beta_ip, ideal_subs_parameters[2]+t0_ip]
        # Calculate log likelihood of this initial proposal for subsidence parameters
        ll += normpdf_ll(ideal_subs_parameters, ideal_subs_parameters_sigma, subs_parameters)

    # STEP 3: calculate log likelihood for the fit of the thermal subsidence curve in the initial proposal
        subsidence_height_t = subsidencebottom .<= model_heights .<= subsidencetop
        ts_model_ages = reverse(model_ages[subsidence_height_t])
        input_subsidence_t = subsidencebottom .<= (-model_strat_heights*1000) .<= subsidencetop
        ts_Sμ = Sμ[input_subsidence_t]
        ts_Sσ = Sσ[input_subsidence_t]
        ll += subsidence_ll(E₀, τ, ts_Sμ, ts_Sσ, ts_model_ages, subs_parameters)/length(ts_Sμ)

    # Preallocate variables for MCMC proposals
        llₚ = ll
        model_agesₚ = copy(model_ages)
        closestₚ = copy(closest)
        sample_heightₚ = copy(sample_height)
        closest_model_agesₚ = copy(closest_model_ages)
        subs_parametersₚ = copy(subs_parameters)

    # Run burnin
        # acceptancedist = fill(false,burnin)
        lldist_burnin = Array{Float64}(undef,burnin÷1000)

        print("Burn-in: ", burnin, " steps\n")
        pgrs = Progress(burnin, desc="Burn-in...")
        pgrs_interval = ceil(Int,sqrt(burnin))
        for n=1:burnin
            # Prepare proposal
            copyto!(model_agesₚ, model_ages)
            copyto!(closestₚ, closest)
            copyto!(sample_heightₚ, sample_height)
            copyto!(subs_parametersₚ, subs_parameters)

            # Propose adjustment to subsidence_parametersₚ
            subs_parametersₚ .+= randn.() .* ideal_subs_parameters_sigma

            if rand() < 0.1
                # Adjust heights
                @inbounds for i=1:length(sample_heightₚ)
                    sample_heightₚ[i] += randn() * Height_sigma[i]
                    closestₚ[i] = round(Int,(sample_heightₚ[i] - model_heights[1])/resolution)+1
                    if closestₚ[i] < 1 # Check we're still within bounds
                        closestₚ[i] = 1
                    elseif closestₚ[i] > npoints
                        closestₚ[i] = npoints
                    end
                end
            else
                # Adjust one point at a time then resolve conflicts
                r = randn() * aveuncert # Generate a random adjustment
                chosen_point = ceil(Int, rand() * npoints) # Pick a point
                model_agesₚ[chosen_point] += r
                #Resolve conflicts
                if r > 0 # If proposing increased age
                    @inbounds for i=1:chosen_point # younger points that are still stratigraphically below
                        if model_agesₚ[i] < model_agesₚ[chosen_point]
                            model_agesₚ[i] = model_agesₚ[chosen_point]
                        end
                    end
                else # if proposing decreased age
                    @inbounds for i=chosen_point:npoints # older points that are still stratigraphically above
                        if model_agesₚ[i] > model_agesₚ[chosen_point]
                            model_agesₚ[i] = model_agesₚ[chosen_point]
                        end
                    end
                end
            end

            # Calculate log likelihood of proposal
            # Proposals younger than age constraint are given a pass if Age_Sidedness is -1 (maximum age)
            # proposal older than age constraint are given a pass if Age_Sidedness is +1 (minimum age)
            @inbounds for i=1:length(Age)
                closest_model_agesₚ[i] = model_agesₚ[closestₚ[i]]
                if Age_Sidedness[i] == sign(closest_model_agesₚ[i] - Age[i])
                    closest_model_agesₚ[i] = Age[i]
                end
            end
            llₚ = normpdf_ll(Age, Age_sigma, closest_model_agesₚ)
            llₚ += normpdf_ll(Height, Height_sigma, sample_heightₚ)
            llₚ += normpdf_ll(ideal_subs_parameters, ideal_subs_parameters_sigma, subs_parametersₚ)

            ts_model_agesₚ = reverse(model_agesₚ[subsidence_height_t])
            llₚ += subsidence_ll(E₀, τ, ts_Sμ, ts_Sσ, ts_model_agesₚ, subs_parametersₚ)/length(ts_Sμ)

            # Accept or reject proposal based on likelihood
            if log(rand(Float64)) < (llₚ - ll)
                ll = llₚ
                copyto!(model_ages, model_agesₚ)
                copyto!(closest, closestₚ)
                copyto!(sample_height, sample_heightₚ)
                copyto!(subs_parameters, subs_parametersₚ)
                # acceptancedist[i] = true
            end
            if mod(n,1000) == 0
                lldist_burnin[n÷1000] = ll
            end

            # Update progress meter every `pgrs_interval` steps
            mod(n,pgrs_interval)==0 && update!(pgrs, n)
        end
        update!(pgrs, burnin) # Finalize

    # Run Markov Chain Monte Carlo
        print("Collecting sieved stationary distribution: ", nsteps*sieve, " steps\n")
        agedist = Array{Float64}(undef,npoints,nsteps)
        lldist = Array{Float64}(undef,nsteps)
        beta_t0dist = Array{Float64}(undef,2,nsteps)

    # Run the model
        pgrs = Progress(nsteps*sieve, desc="Collecting...")
        pgrs_interval = ceil(Int,sqrt(nsteps*sieve))
        for n=1:(nsteps*sieve)
            # Prepare proposal
            copyto!(model_agesₚ, model_ages)
            copyto!(closestₚ, closest)
            copyto!(sample_heightₚ, sample_height)
            copyto!(subs_parametersₚ, subs_parameters)

            # Propose adjustment to subsidence_parametersₚ
            subs_parametersₚ .+= randn.() .*ideal_subs_parameters_sigma

            if rand() < 0.1
                # Adjust heights
                @inbounds for i=1:length(sample_heightₚ)
                    sample_heightₚ[i] += randn() * Height_sigma[i]
                    closestₚ[i] = round(Int,(sample_heightₚ[i] - model_heights[1])/resolution)+1
                    if closestₚ[i] < 1 # Check we're still within bounds
                        closestₚ[i] = 1
                    elseif closestₚ[i] > npoints
                        closestₚ[i] = npoints
                    end
                end
            else
                # Adjust one point at a time then resolve conflicts
                r = randn() * aveuncert # Generate a random adjustment
                chosen_point = ceil(Int, rand() * npoints) # Pick a point
                model_agesₚ[chosen_point] += r
                #Resolve conflicts
                if r > 0 # If proposing increased age
                    @inbounds for i=1:chosen_point # younger points below
                        if model_agesₚ[i] < model_agesₚ[chosen_point]
                            model_agesₚ[i] = model_agesₚ[chosen_point]
                        end
                    end
                else # if proposing decreased age
                    @inbounds for i=chosen_point:npoints # older points above
                        if model_agesₚ[i] > model_agesₚ[chosen_point]
                            model_agesₚ[i] = model_agesₚ[chosen_point]
                        end
                    end
                end
            end

            # Calculate log likelihood of proposal
            # Proposals younger than age constraint are given a pass if Age_Sidedness is -1 (maximum age)
            # proposal older than age constraint are given a pass if Age_Sidedness is +1 (minimum age)
            @inbounds for i=1:length(Age)
                closest_model_agesₚ[i] = model_agesₚ[closestₚ[i]]
                if Age_Sidedness[i] == sign(closest_model_agesₚ[i] - Age[i])
                    closest_model_agesₚ[i] = Age[i]
                end
            end
            llₚ = normpdf_ll(Age, Age_sigma, closest_model_agesₚ)
            llₚ += normpdf_ll(Height, Height_sigma, sample_heightₚ)
            llₚ += normpdf_ll(ideal_subs_parameters, ideal_subs_parameters_sigma, subs_parametersₚ)

            ts_model_agesₚ = reverse(model_agesₚ[subsidence_height_t])
            llₚ += subsidence_ll(E₀, τ, ts_Sμ, ts_Sσ, ts_model_agesₚ, subs_parametersₚ)/length(ts_Sμ)

            # Accept or reject proposal based on likelihood
            if log(rand(Float64)) < (llₚ - ll)
                ll = llₚ
                copyto!(model_ages, model_agesₚ)
                copyto!(closest, closestₚ)
                copyto!(sample_height, sample_heightₚ)
                copyto!(subs_parameters, subs_parametersₚ)
            end

            # Record sieved results
            if mod(n,sieve) == 0
                lldist[n÷sieve] = ll
                agedist[:,n÷sieve] .= model_ages
                beta_t0dist[:,n÷sieve] .= subs_parameters
                #predicted_ages[n÷sieve] = beta_t0dist[2]+τ*log(1-(Sμ[?]*pi)/(E₀*beta_t0dist[1]*sin(pi/beta_t0dist[2])))
            end

            # Update progress meter every `pgrs_interval` steps
            mod(n,pgrs_interval)==0 && update!(pgrs, n)
        end
        update!(pgrs,nsteps*sieve)

    # Crop the result
        agedist = agedist[active_height_t,:]
        subsmdl = SubsidenceStratAgeModel(
            model_heights[active_height_t], # Model heights
            nanmean(agedist,dim=2), # Mean age
            nanstd(agedist,dim=2), # Standard deviation
            nanmedian(agedist,dim=2), # Median age
            nanpctile(agedist,2.5,dim=2), # 2.5th percentile
            nanpctile(agedist,97.5,dim=2), # 97.5th percentile
            nanmean(beta_t0dist[1,:],dim=1), # Mean beta
            nanstd(beta_t0dist[1,:],dim=1), # Standard deviation
            nanmedian(beta_t0dist[1,:],dim=1), # Median beta
            nanpctile(beta_t0dist[1,:],2.5,dim=1), # 2.5th percentile
            nanpctile(beta_t0dist[1,:],97.5,dim=1), # 97.5th percentile
            nanmean(beta_t0dist[2,:],dim=1), # Mean T0
            nanstd(beta_t0dist[2,:],dim=1), # Standard deviation
            nanmedian(beta_t0dist[2,:],dim=1), # Median T0
            nanpctile(beta_t0dist[2,:],2.5,dim=1), # 2.5th percentile
            nanpctile(beta_t0dist[2,:],97.5,dim=1) # 97.5th percentile
        )
    return subsmdl, agedist, lldist, beta_t0dist, lldist_burnin
end

## Parameterize age-height model with age control

In [ ]:
#Input the number of samples to be used in model, must match below.
    nSamples = 12 #SEE BELOW
# Make an instance of a ChronSection object for nSamples
    smpl = NewChronAgeData(nSamples)
    smpl.Name          = ("BaseTCU1","Base-TCU3_base-Sturt","TCU4-1_MDA","Top-TCU5_end_Sturt","TC7carbs","Base-TC7diamict_base_Marinoan","Base-Yelland_end_Marinoan",
                          "Mid-Osceola_Shuram_chemoCorr","Pioche_base","Middle_Dunderberg","Top_Pogonip","Top_ElyLimestone") #Strat_height_name
    smpl.Age           = [750,717.4,688.5,661,651.7,640,635.3,570,507,493,469,315.2] # Strat_age, MEASURED
    smpl.Age_sigma     = [5,0.1,2.4,0.2,0.6,2,1.1,3,3,0.5,2,0.4]/2 # 2-σ age uncertainties, divided by 2; chron.jl uses 1sigma uncertainties as input data
    smpl.Height[:]     = [-13189,-13020,-12670,-12260,-10159,-9760,-9660,-6910,-5350,-3745,-2155,0] # Strat height in generalized Trout Creek-McCoy Creek Group section
    smpl.Height_sigma[:]  = [0,0,0,0,0,0,0,0,0,0,0,0] # Strat height uncertainty: initial try at 10m minimum for generalized section. 
    smpl.Age_Sidedness[:] = [0,0,-1,0,-1,0,0,0,0,0,0,1] # Sidedness (zeros by default: geochron constraints are two-sided). Use -1 for a maximum age and +1 for a minimum age, 0 for two-sided

 

    # # # # # # # # # # Enter thermal subsidence parameter priors here! # # # # # # # # # #
    # Enter initial guesses for the beta factor and thermal subsidence onset age and their uncertainties
    Beta = 5.5
    Beta_sigma = 1
    T0 = 655
    T0_sigma = 40

    therm = NewThermalSubsidenceParameters()
    therm.Param = [Beta, T0]
    therm.Sigma = [Beta_sigma, T0_sigma]
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

In [ ]:
# # # # # # # # # # Configure MCMC model here! # # # # # # # # # #
    # Configure the stratigraphic MCMC model
    config = NewStratAgeModelConfiguration()
    # If in doubt, you can probably leave these parameters as-is
    config.resolution = res*1000 # Same units as sample height. Smaller is slower!
    config.bounding = 1.0 # how far away do we place runaway bounds, as a fraction of total section height. Larger is slower.
    (bottom, top) = extrema(smpl.Height)
    npoints_approx = round(Int,length(bottom:config.resolution:top) * (1 + 2*config.bounding))
    config.nsteps = 7000 # Number of steps to run in distribution MCMC 
    config.burnin = 1000*npoints_approx # Number to discard 
    config.sieve = round(Int,npoints_approx) # Record one out of every nsieve steps
    # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

    # Let's see if adding just a little bit of systematic error on tectonic subsidence solves the ll = -Inf problem
    # Currently the systematic error is set to be = 1% of the sigma for the tectonic subsidence of the first (oldest) layer
    Sσ_corr = Sσ .+ ((Sμ[end-1]-Sμ[end])/100)


        #Run the model
        (subsmdl, agedist, lldist, beta_t0dist, lldist_burnin) = SubsidenceStratMetropolis(smpl, config, therm, model_strat_heights, Sμ, Sσ, Beta_sigma/10, T0_sigma/10)

        #= Code for storing and reading age-depth model results
        # Store and read results
        writedlm("agedist2.csv", agedist2, ",")
        writedlm("lldist2.csv", lldist2)
        writedlm("beta_t0dist2.csv", beta_t0dist2, ",")
        writedlm("lldist_burnin2.csv", lldist_burnin2)

        agedist3 = readdlm("agedist3.csv", ',', Float64)
        lldist3 = readdlm("lldist3.csv", ',', Float64)
        beta_t0dist3 = readdlm("beta_t0dist3.csv", ',', Float64)
        lldist_burnin3 = readdlm("lldist_burnin3.csv", ',', Float64)
        =#

In [ ]:
 # Plot 1: Age-depth model (mean and 95% confidence interval for both model and data)
        hdl = plot([subsmdl.Age_025CI; reverse(subsmdl.Age_975CI)],[subsmdl.Height; reverse(subsmdl.Height)], fill=(round(Int,minimum(subsmdl.Height)),0.5,:blue), label="model")
        plot!(hdl, subsmdl.Age, subsmdl.Height, linecolor=:blue, label="", fg_color_legend=:white) # Center line
        t = smpl.Age_Sidedness .== 0 # Two-sided constraints (plot in black)
        any(t) && plot!(hdl, smpl.Age[t], smpl.Height[t], xerror=(2*smpl.Age_sigma[t]),yerror=(2*smpl.Height_sigma[t]),label="data",seriestype=:scatter,color=:black)
        t = smpl.Age_Sidedness .== 1 # Minimum ages (plot in cyan)
        any(t) && plot!(hdl, smpl.Age[t], smpl.Height[t], xerror=(2*smpl.Age_sigma[t],zeros(count(t))),label="",seriestype=:scatter,color=:cyan,msc=:cyan)
        any(t) && zip(smpl.Age[t], smpl.Age[t].+nanmean(smpl.Age_sigma[t])*4, smpl.Height[t]) .|> x-> plot!([x[1],x[2]],[x[3],x[3]], arrow=true, label="", color=:cyan)
        t = smpl.Age_Sidedness .== -1 # Maximum ages (plot in orange)
        any(t) && plot!(hdl, smpl.Age[t], smpl.Height[t], xerror=(zeros(count(t)),2*smpl.Age_sigma[t]),label="",seriestype=:scatter,color=:orange,msc=:orange)
        any(t) && zip(smpl.Age[t], smpl.Age[t].-nanmean(smpl.Age_sigma[t])*4, smpl.Height[t]) .|> x-> plot!([x[1],x[2]],[x[3],x[3]], arrow=true, label="", color=:orange)
        plot!(hdl, xlabel="Age ($(smpl.Age_Unit))", ylabel="Height ($(smpl.Height_Unit))")
        #plot!(hdl, [smpl.Age[1], smpl.Height[1]],[smpl.Age[3], smpl.Height[3]])
        savefig(hdl,Path*"McCoy_Age_Height.pdf")
        display(hdl)

In [ ]:
## Pair decompaction model increments with age-height model increments:

In [ ]:
height = [0
-10
-20
-30
-40
-50
-60
-70
-80
-90
-100
-110
-120
-130
-140
-150
-160
-170
-180
-190
-200
-210
-220
-230
-240
-250
-260
-270
-280
-290
-300
-310
-320
-330
-340
-350
-360
-370
-380
-390
-400
-410
-420
-430
-440
-450
-460
-470
-480
-490
-500
-510
-520
-530
-540
-550
-560
-570
-580
-590
-600
-610
-620
-630
-640
-650
-660
-670
-680
-690
-700
-710
-720
-730
-740
-750
-760
-770
-780
-790
-800
-810
-820
-830
-840
-850
-860
-870
-880
-890
-900
-910
-920
-930
-940
-950
-960
-970
-980
-990
-1000
-1010
-1020
-1030
-1040
-1050
-1060
-1070
-1080
-1090
-1100
-1110
-1120
-1130
-1140
-1150
-1160
-1170
-1180
-1190
-1200
-1210
-1220
-1230
-1240
-1250
-1260
-1270
-1280
-1290
-1300
-1310
-1320
-1330
-1340
-1350
-1360
-1370
-1380
-1390
-1400
-1410
-1420
-1430
-1440
-1450
-1460
-1470
-1480
-1490
-1500
-1510
-1520
-1530
-1540
-1550
-1560
-1570
-1580
-1590
-1600
-1610
-1620
-1630
-1640
-1650
-1660
-1670
-1680
-1690
-1700
-1710
-1720
-1730
-1740
-1750
-1760
-1770
-1780
-1790
-1800
-1810
-1820
-1830
-1840
-1850
-1860
-1870
-1880
-1890
-1900
-1910
-1920
-1930
-1940
-1950
-1960
-1970
-1980
-1990
-2000
-2010
-2020
-2030
-2040
-2050
-2060
-2070
-2080
-2090
-2100
-2110
-2120
-2130
-2140
-2150
-2160
-2170
-2180
-2190
-2200
-2210
-2220
-2230
-2240
-2250
-2260
-2270
-2280
-2290
-2300
-2310
-2320
-2330
-2340
-2350
-2360
-2370
-2380
-2390
-2400
-2410
-2420
-2430
-2440
-2450
-2460
-2470
-2480
-2490
-2500
-2510
-2520
-2530
-2540
-2550
-2560
-2570
-2580
-2590
-2600
-2610
-2620
-2630
-2640
-2650
-2660
-2670
-2680
-2690
-2700
-2710
-2720
-2730
-2740
-2750
-2760
-2770
-2780
-2790
-2800
-2810
-2820
-2830
-2840
-2850
-2860
-2870
-2880
-2890
-2900
-2910
-2920
-2930
-2940
-2950
-2960
-2970
-2980
-2990
-3000
-3010
-3020
-3030
-3040
-3050
-3060
-3070
-3080
-3090
-3100
-3110
-3120
-3130
-3140
-3150
-3160
-3170
-3180
-3190
-3200
-3210
-3220
-3230
-3240
-3250
-3260
-3270
-3280
-3290
-3300
-3310
-3320
-3330
-3340
-3350
-3360
-3370
-3380
-3390
-3400
-3410
-3420
-3430
-3440
-3450
-3460
-3470
-3480
-3490
-3500
-3510
-3520
-3530
-3540
-3550
-3560
-3570
-3580
-3590
-3600
-3610
-3620
-3630
-3640
-3650
-3660
-3670
-3680
-3690
-3700
-3710
-3720
-3730
-3740
-3750
-3760
-3770
-3780
-3790
-3800
-3810
-3820
-3830
-3840
-3850
-3860
-3870
-3880
-3890
-3900
-3910
-3920
-3930
-3940
-3950
-3960
-3970
-3980
-3990
-4000
-4010
-4020
-4030
-4040
-4050
-4060
-4070
-4080
-4090
-4100
-4110
-4120
-4130
-4140
-4150
-4160
-4170
-4180
-4190
-4200
-4210
-4220
-4230
-4240
-4250
-4260
-4270
-4280
-4290
-4300
-4310
-4320
-4330
-4340
-4350
-4360
-4370
-4380
-4390
-4400
-4410
-4420
-4430
-4440
-4450
-4460
-4470
-4480
-4490
-4500
-4510
-4520
-4530
-4540
-4550
-4560
-4570
-4580
-4590
-4600
-4610
-4620
-4630
-4640
-4650
-4660
-4670
-4680
-4690
-4700
-4710
-4720
-4730
-4740
-4750
-4760
-4770
-4780
-4790
-4800
-4810
-4820
-4830
-4840
-4850
-4860
-4870
-4880
-4890
-4900
-4910
-4920
-4930
-4940
-4950
-4960
-4970
-4980
-4990
-5000
-5010
-5020
-5030
-5040
-5050
-5060
-5070
-5080
-5090
-5100
-5110
-5120
-5130
-5140
-5150
-5160
-5170
-5180
-5190
-5200
-5210
-5220
-5230
-5240
-5250
-5260
-5270
-5280
-5290
-5300
-5310
-5320
-5330
-5340
-5350
-5360
-5370
-5380
-5390
-5400
-5410
-5420
-5430
-5440
-5450
-5460
-5470
-5480
-5490
-5500
-5510
-5520
-5530
-5540
-5550
-5560
-5570
-5580
-5590
-5600
-5610
-5620
-5630
-5640
-5650
-5660
-5670
-5680
-5690
-5700
-5710
-5720
-5730
-5740
-5750
-5760
-5770
-5780
-5790
-5800
-5810
-5820
-5830
-5840
-5850
-5860
-5870
-5880
-5890
-5900
-5910
-5920
-5930
-5940
-5950
-5960
-5970
-5980
-5990
-6000
-6010
-6020
-6030
-6040
-6050
-6060
-6070
-6080
-6090
-6100
-6110
-6120
-6130
-6140
-6150
-6160
-6170
-6180
-6190
-6200
-6210
-6220
-6230
-6240
-6250
-6260
-6270
-6280
-6290
-6300
-6310
-6320
-6330
-6340
-6350
-6360
-6370
-6380
-6390
-6400
-6410
-6420
-6430
-6440
-6450
-6460
-6470
-6480
-6490
-6500
-6510
-6520
-6530
-6540
-6550
-6560
-6570
-6580
-6590
-6600
-6610
-6620
-6630
-6640
-6650
-6660
-6670
-6680
-6690
-6700
-6710
-6720
-6730
-6740
-6750
-6760
-6770
-6780
-6790
-6800
-6810
-6820
-6830
-6840
-6850
-6860
-6870
-6880
-6890
-6900
-6910
-6920
-6930
-6940
-6950
-6960
-6970
-6980
-6990
-7000
-7010
-7020
-7030
-7040
-7050
-7060
-7070
-7080
-7090
-7100
-7110
-7120
-7130
-7140
-7150
-7160
-7170
-7180
-7190
-7200
-7210
-7220
-7230
-7240
-7250
-7260
-7270
-7280
-7290
-7300
-7310
-7320
-7330
-7340
-7350
-7360
-7370
-7380
-7390
-7400
-7410
-7420
-7430
-7440
-7450
-7460
-7470
-7480
-7490
-7500
-7510
-7520
-7530
-7540
-7550
-7560
-7570
-7580
-7590
-7600
-7610
-7620
-7630
-7640
-7650
-7660
-7670
-7680
-7690
-7700
-7710
-7720
-7730
-7740
-7750
-7760
-7770
-7780
-7790
-7800
-7810
-7820
-7830
-7840
-7850
-7860
-7870
-7880
-7890
-7900
-7910
-7920
-7930
-7940
-7950
-7960
-7970
-7980
-7990
-8000
-8010
-8020
-8030
-8040
-8050
-8060
-8070
-8080
-8090
-8100
-8110
-8120
-8130
-8140
-8150
-8160
-8170
-8180
-8190
-8200
-8210
-8220
-8230
-8240
-8250
-8260
-8270
-8280
-8290
-8300
-8310
-8320
-8330
-8340
-8350
-8360
-8370
-8380
-8390
-8400
-8410
-8420
-8430
-8440
-8450
-8460
-8470
-8480
-8490
-8500
-8510
-8520
-8530
-8540
-8550
-8560
-8570
-8580
-8590
-8600
-8610
-8620
-8630
-8640
-8650
-8660
-8670
-8680
-8690
-8700
-8710
-8720
-8730
-8740
-8750
-8760
-8770
-8780
-8790
-8800
-8810
-8820
-8830
-8840
-8850
-8860
-8870
-8880
-8890
-8900
-8910
-8920
-8930
-8940
-8950
-8960
-8970
-8980
-8990
-9000
-9010
-9020
-9030
-9040
-9050
-9060
-9070
-9080
-9090
-9100
-9110
-9120
-9130
-9140
-9150
-9160
-9170
-9180
-9190
-9200
-9210
-9220
-9230
-9240
-9250
-9260
-9270
-9280
-9290
-9300
-9310
-9320
-9330
-9340
-9350
-9360
-9370
-9380
-9390
-9400
-9410
-9420
-9430
-9440
-9450
-9460
-9470
-9480
-9490
-9500
-9510
-9520
-9530
-9540
-9550
-9560
-9570
-9580
-9590
-9600
-9610
-9620
-9630
-9640
-9650
-9660
-9670
-9680
-9690
-9700
-9710
-9720
-9730
-9740
-9750
-9760
-9770
-9780
-9790
-9800
-9810
-9820
-9830
-9840
-9850
-9860
-9870
-9880
-9890
-9900
-9910
-9920
-9930
-9940
-9950
-9960
-9970
-9980
-9990
-10000
-10010
-10020
-10030
-10040
-10050
-10060
-10070
-10080
-10090
-10100
-10110
-10120
-10130
-10140
-10150
-10160
-10170
-10180
-10190
-10200
-10210
-10220
-10230
-10240
-10250
-10260
-10270
-10280
-10290
-10300
-10310
-10320
-10330
-10340
-10350
-10360
-10370
-10380
-10390
-10400
-10410
-10420
-10430
-10440
-10450
-10460
-10470
-10480
-10490
-10500
-10510
-10520
-10530
-10540
-10550
-10560
-10570
-10580
-10590
-10600
-10610
-10620
-10630
-10640
-10650
-10660
-10670
-10680
-10690
-10700
-10710
-10720
-10730
-10740
-10750
-10760
-10770
-10780
-10790
-10800
-10810
-10820
-10830
-10840
-10850
-10860
-10870
-10880
-10890
-10900
-10910
-10920
-10930
-10940
-10950
-10960
-10970
-10980
-10990
-11000
-11010
-11020
-11030
-11040
-11050
-11060
-11070
-11080
-11090
-11100
-11110
-11120
-11130
-11140
-11150
-11160
-11170
-11180
-11190
-11200
-11210
-11220
-11230
-11240
-11250
-11260
-11270
-11280
-11290
-11300
-11310
-11320
-11330
-11340
-11350
-11360
-11370
-11380
-11390
-11400
-11410
-11420
-11430
-11440
-11450
-11460
-11470
-11480
-11490
-11500
-11510
-11520
-11530
-11540
-11550
-11560
-11570
-11580
-11590
-11600
-11610
-11620
-11630
-11640
-11650
-11660
-11670
-11680
-11690
-11700
-11710
-11720
-11730
-11740
-11750
-11760
-11770
-11780
-11790
-11800
-11810
-11820
-11830
-11840
-11850
-11860
-11870
-11880
-11890
-11900
-11910
-11920
-11930
-11940
-11950
-11960
-11970
-11980
-11990
-12000
-12010
-12020
-12030
-12040
-12050
-12060
-12070
-12080
-12090
-12100
-12110
-12120
-12130
-12140
-12150
-12160
-12170
-12180
-12190
-12200
-12210
-12220
-12230
-12240
-12250
-12260
-12270
-12280
-12290
-12300
-12310
-12320
-12330
-12340
-12350
-12360
-12370
-12380
-12390
-12400
-12410
-12420
-12430
-12440
-12450
-12460
-12470
-12480
-12490
-12500
-12510
-12520
-12530
-12540
-12550
-12560
-12570
-12580
-12590
-12600
-12610
-12620
-12630
-12640
-12650
-12660
-12670
-12680
-12690
-12700
-12710
-12720
-12730
-12740
-12750
-12760
-12770
-12780
-12790
-12800
-12810
-12820
-12830
-12840
-12850
-12860
-12870
-12880
-12890
-12900
-12910
-12920
-12930
-12940
-12950
-12960
-12970
-12980
-12990
-13000
-13010
-13020
-13030
-13040
-13050
-13060
-13070
-13080
-13090
-13100
-13110
-13120
-13130
-13140
-13150
-13160
-13170
-13180
]

single_age_at_height = linterp1s(subsmdl.Height,subsmdl.Age,height)
spec_age = linterp1s(subsmdl.Height,subsmdl.Age,[-11660])
writedlm(Path*"Age_at_height_McCoy.csv", single_age_at_height, ',')

display(spec_age) 

In [ ]:
 # Plot results - tectonic subsidence in comparison with present day stratigraphic heights
    age=single_age_at_height
    p = plot(age, Sμ, yflip=true, xflip=true, color="navy",alpha=.01)
    
    plot!(p,(single_age_at_height),Sₜ[:,2:end], alpha = 0.01,yflip = true, xflip=true, color = "navy")
    


    display(p)
savefig(p, Path*"McCoy_Full_Adj_highres_SL.pdf")

In [ ]:
# Set bin width and spacing
#binwidth = round(nanrange(mdl.Age)/10,sigdigits=1) # Can also set manually, commented out below
binwidth = round(nanrange(subsmdl.Age)/30,sigdigits=1) # Can also set manually, commented out below
#binwidth = 5# Same units as smpl.Age
binoverlap = 10
ages = collect(minimum(subsmdl.Age):binwidth/binoverlap:maximum(subsmdl.Age))
bincenters = ages[1+Int(binoverlap/2):end-Int(binoverlap/2)]
spacing = binoverlap

# Calculate rates for the stratigraphy of each markov chain step
dhdt_dist = Array{Float64}(undef, length(ages)-binoverlap, config.nsteps)
@time for i=1:config.nsteps
    heights = linterp1(reverse(agedist[:,i]), reverse(subsmdl.Height), ages)
    dhdt_dist[:,i] .= abs.(heights[1:end-spacing] - heights[spacing+1:end]) ./ binwidth
end

# Find mean and 1-sigma (68%) CI
dhdt = nanmean(dhdt_dist,dim=2)
dhdt_50p = nanmedian(dhdt_dist,dim=2)
dhdt_16p = nanpctile(dhdt_dist,15.865,dim=2) # Minus 1-sigma (15.865th percentile)
dhdt_84p = nanpctile(dhdt_dist,84.135,dim=2) # Plus 1-sigma (84.135th percentile)
# Other confidence intervals (10:10:50)
dhdt_20p = nanpctile(dhdt_dist,20,dim=2)
dhdt_80p = nanpctile(dhdt_dist,80,dim=2)
dhdt_25p = nanpctile(dhdt_dist,25,dim=2)
dhdt_75p = nanpctile(dhdt_dist,75,dim=2)
dhdt_30p = nanpctile(dhdt_dist,30,dim=2)
dhdt_70p = nanpctile(dhdt_dist,70,dim=2)
dhdt_35p = nanpctile(dhdt_dist,35,dim=2)
dhdt_65p = nanpctile(dhdt_dist,65,dim=2)
dhdt_40p = nanpctile(dhdt_dist,40,dim=2)
dhdt_60p = nanpctile(dhdt_dist,60,dim=2)
dhdt_45p = nanpctile(dhdt_dist,45,dim=2)
dhdt_55p = nanpctile(dhdt_dist,55,dim=2)

# Plot results
hdl = plot(bincenters,dhdt, label="Mean", color=:black, linewidth=2)
plot!(hdl,[bincenters; reverse(bincenters)],[dhdt_16p; reverse(dhdt_84p)], fill=(0,0.2,:darkblue), linealpha=0, label="68% CI")
plot!(hdl,[bincenters; reverse(bincenters)],[dhdt_20p; reverse(dhdt_80p)], fill=(0,0.2,:darkblue), linealpha=0, label="")
plot!(hdl,[bincenters; reverse(bincenters)],[dhdt_25p; reverse(dhdt_75p)], fill=(0,0.2,:darkblue), linealpha=0, label="")
plot!(hdl,[bincenters; reverse(bincenters)],[dhdt_30p; reverse(dhdt_70p)], fill=(0,0.2,:darkblue), linealpha=0, label="")
plot!(hdl,[bincenters; reverse(bincenters)],[dhdt_35p; reverse(dhdt_65p)], fill=(0,0.2,:darkblue), linealpha=0, label="")
plot!(hdl,[bincenters; reverse(bincenters)],[dhdt_40p; reverse(dhdt_60p)], fill=(0,0.2,:darkblue), linealpha=0, label="")
plot!(hdl,[bincenters; reverse(bincenters)],[dhdt_45p; reverse(dhdt_55p)], fill=(0,0.2,:darkblue), linealpha=0, label="")
plot!(hdl,bincenters,dhdt_50p, label="Median", color=:grey, linewidth=1)
plot!(hdl, xlabel="Age ($(smpl.Age_Unit))", ylabel="Depositional Rate ($(smpl.Height_Unit) / $(smpl.Age_Unit))", fg_color_legend=:white)
vline!(hdl,[635,640,660])
display(hdl)


writedlm(Path*"McCoy_bincenters.csv", bincenters, ',')
writedlm(Path*"McCoy_dhdt.csv", dhdt, ',')
writedlm(Path*"McCoy_dhdt_50.csv", dhdt_50p, ',')
writedlm(Path*"McCoy_dhdt_16.csv", dhdt_16p, ',')
writedlm(Path*"McCoy_dhdt_84.csv", dhdt_84p, ',')
writedlm(Path*"McCoy_dhdt_20.csv", dhdt_20p, ',')
writedlm(Path*"McCoy_dhdt_25.csv", dhdt_25p, ',')
writedlm(Path*"McCoy_dhdt_30.csv", dhdt_30p, ',')
writedlm(Path*"McCoy_dhdt_35.csv", dhdt_35p, ',')
writedlm(Path*"McCoy_dhdt_40.csv", dhdt_40p, ',')
writedlm(Path*"McCoy_dhdt_45.csv", dhdt_45p, ',')
writedlm(Path*"McCoy_dhdt_55.csv", dhdt_55p, ',')
writedlm(Path*"McCoy_dhdt_60.csv", dhdt_60p, ',')
writedlm(Path*"McCoy_dhdt_65.csv", dhdt_65p, ',')
writedlm(Path*"McCoy_dhdt_70.csv", dhdt_70p, ',')
writedlm(Path*"McCoy_dhdt_75.csv", dhdt_75p, ',')
writedlm(Path*"McCoy_dhdt_80.csv", dhdt_80p, ',')